In [1]:
import pandas as pd
import numpy as np
import hashlib
import json
import math
import sklearn
import seaborn as sns 
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.linear_model import Ridge
import urllib.request as ur
import my_utils
import data_understanding
import data_prep
import data_model

# Author : Rute Ayalew
# Date: Oct 7, 2023
# Collaborators: Chat GPT

In [2]:
def parseData(fname):
  for l in ur.urlopen(fname):
    yield eval(l)

In [3]:
print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("Done.")

Reading data...
Done.


In [4]:
df = pd.DataFrame.from_dict(data, orient='columns')
df

,review/appearance,beer/style,review/palate,review/taste,beer/name,review/timeUnix,beer/ABV,beer/beerId,beer/brewerId,review/timeStruct,review/overall,review/text,user/profileName,review/aroma,user/gender,user/birthdayRaw,user/birthdayUnix,user/ageInSeconds
0,2.5,Hefeweizen,1.5,1.5,Sausa Weizen,1234817823,5.0,47986,10325,"{'isdst': 0, 'mday': 16, 'hour': 20, 'min': 57...",1.5,A lot of foam. But a lot.\tIn the smell some b...,stcules,2.0,NaN,NaN,NaN,NaN
1,3.0,English Strong Ale,3.0,3.0,Red Moon,1235915097,6.2,48213,10325,"{'isdst': 0, 'mday': 1, 'hour': 13, 'min': 44,...",3.0,"Dark red color, light beige foam, average.\tIn...",stcules,2.5,NaN,NaN,NaN,NaN
2,3.0,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,1235916604,6.5,48215,10325,"{'isdst': 0, 'mday': 1, 'hour': 14, 'min': 10,...",3.0,"Almost totally black. Beige foam, quite compac...",stcules,2.5,NaN,NaN,NaN,NaN
3,3.5,German Pilsener,2.5,3.0,Sausa Pils,1234725145,5.0,47969,10325,"{'isdst': 0, 'mday': 15, 'hour': 19, 'min': 12...",3.0,"Golden yellow color. White, compact foam, quit...",stcules,3.0,NaN,NaN,NaN,NaN
4,4.0,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,1293735206,7.7,64883,1075,"{'isdst': 0, 'mday': 30, 'hour': 18, 'min': 53...",4.0,"According to the website, the style for the Ca...",johnmichaelsen,4.5,Male,"Jun 16, 1901",-2.163082e+09,3.581417e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4.0,American Double / Imperial Stout,4.0,4.0,Stoudt's Fat Dog (Imperial Oatmeal Stout),1196810078,9.0,20539,394,"{'isdst': 0, 'mday': 4, 'hour': 23, 'min': 14,...",4.0,Bottled on 5/16/2007 and poured into an Imperi...,Jwale73,4.0,NaN,NaN,NaN,NaN
49996,3.5,American Double / Imperial Stout,4.0,4.0,Stoudt's Fat Dog (Imperial Oatmeal Stout),1196388069,9.0,20539,394,"{'isdst': 0, 'mday': 30, 'hour': 2, 'min': 1, ...",4.0,The first beer I put away for cellaring. 12 oz...,bigeyedfish,4.0,NaN,NaN,NaN,NaN
49997,4.0,American Double / Imperial Stout,4.0,4.5,Stoudt's Fat Dog (Imperial Oatmeal Stout),1196218959,9.0,20539,394,"{'isdst': 0, 'mday': 28, 'hour': 3, 'min': 2, ...",3.5,12 oz. bottle sampled in a La Fin du Monde tul...,stewdawg,3.5,NaN,NaN,NaN,NaN
49998,4.0,American Double / Imperial Stout,4.0,4.5,Stoudt's Fat Dog (Imperial Oatmeal Stout),1196138077,9.0,20539,394,"{'isdst': 0, 'mday': 27, 'hour': 4, 'min': 34,...",4.0,"I'm on my second case of Fat Dog, so you kind ...",On2Wheels,4.0,NaN,NaN,NaN,NaN


# TASK 1: PHASE II

The following 2 function calls are manual input required to save and log target attribute 
and format of data (whether rows=samples & columns=attributes/feature or vice versa). 

In [5]:
# User sets the target_attribute variable to be saved as a string object and logged
# Argument passed should be the target attribute's string literal
target_attribute = data_understanding.set_target_attribute('review/overall')

Target attribute saved as:  review/overall


Next function call reports a general description of the dataframe

In [6]:
# Print report of dataframe description with format, dimensionality, sample count, attribute count and datatype distribution
# parameters are dataframe, row_ def, and column_def which should recieve associated values of 'samples', and 'attributes' or 'features' to indicate format
data_understanding.report_description(df, 'samples', 'attributes')

GENERATING INITIAL DESCRIPTION OF DATA:

Data dimensions: (50000, 18)
Data has proper formatting with rows=samples and columns=attributes

Sample Count = 
50000
Attribute Count = 
18

Data types that are present:
float64
object
int64


The assignment requested Task 1 to be performed on only numeric attributes thus the remaining operations will be performed on a numeric copy of the data frame named df_numeric

In [7]:
#return and save numeric dataframe
df_numeric = my_utils.numeric_only(df)
my_utils.save_to_csv(df_numeric, '../output', 'numeric_data.csv')
df_numeric

INFO:root:Successfully created file numeric_data.csv in ../output


<<<<<<<<<< Processed data saved to ../output/numeric_data.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/birthdayUnix,user/ageInSeconds
0,2.5,1.5,1.5,1234817823,5.0,1.5,2.0,NaN,NaN
1,3.0,3.0,3.0,1235915097,6.2,3.0,2.5,NaN,NaN
2,3.0,3.0,3.0,1235916604,6.5,3.0,2.5,NaN,NaN
3,3.5,2.5,3.0,1234725145,5.0,3.0,3.0,NaN,NaN
4,4.0,4.0,4.5,1293735206,7.7,4.0,4.5,-2.163082e+09,3.581417e+09
...,...,...,...,...,...,...,...,...,...
49995,4.0,4.0,4.0,1196810078,9.0,4.0,4.0,NaN,NaN
49996,3.5,4.0,4.0,1196388069,9.0,4.0,4.0,NaN,NaN
49997,4.0,4.0,4.5,1196218959,9.0,3.5,3.5,NaN,NaN
49998,4.0,4.0,4.5,1196138077,9.0,4.0,4.0,NaN,NaN


The next step is calling the quality checker function and for this case, the threshold is set at 2 standard deviations because not enough data was found with a threshold of 3 standard deviations.

In [8]:
data_understanding.report_quality(df_numeric)

Number of duplicate rows:  0
Number of rows with missing values: 39521

Identifying rows with out-of-domain properties:
Threshold = 2  standard deviations
Number of rows with out-of-domain properties:  3313


Then I check the skew of the numeric data to inform my decision to either normalize or standardize since the latter is preferred when data follows normal distribution. 

In [9]:
data_understanding.skew_report(df_numeric)

Skewness report:
Number of attributes with left skew:  7
Number of attributes with right skew:  2
Number of attributes with no skew/ are symmetric:  0


Ratio of total skewed attributes to symmetric attributes =  9 : 0
Therefor, the data is mostly skewed


Thus I conclude that normalization is best for Phase III

NOTE: This next step of checking for collinearity was only performed after already building the first 2 models. It crossed my mind as I was thinking about feature selection and its low impact on the linear regression model's evaluation. So I decided to come back to Phase I and check for collinearity. And thus I realized that the user birthday and user age (EXCEPTIONALLY) seems to have a strong collinear relationship. I honestly dont know why user age in seconds is in the 200s for its VIF result when the other feature I assume it to share that collinearity with is only in the 30's. This was interesting to note and I'll try to check for this aspect more in future assignments (despite not being shocking that birthday is related to age lol)

In [10]:
data_understanding.check_collinearity(df_numeric, target_attribute)

             feature         VIF
0  review/appearance    1.569493
1      review/palate    2.279781
2       review/taste    2.723768
3    review/timeUnix    1.087116
4           beer/ABV    1.295594
5       review/aroma    2.222681
6  user/birthdayUnix   36.460001
7  user/ageInSeconds  286.430136


# TASK 1: PHASE III

In [11]:
#dropping null samples and saving numeric data with nulls removed to output

df_numeric_no_null = data_prep.drop_null(df_numeric)
my_utils.save_to_csv(df_numeric_no_null,'../output', 'df_numeric_no_null.csv' )

#dropping rows with out-of-bounds properties and saving reduced numeric data to output

df_numeric_reduced = pd.DataFrame(data_prep.drop_out_of_domain(df_numeric_no_null))
my_utils.save_to_csv(df_numeric_reduced,'../output', 'df_numeric_reduced.csv' )
df_numeric_reduced


Null row removal:
Sample count before:  50000
Sample count after:  10479


INFO:root:Successfully created file df_numeric_no_null.csv in ../output


<<<<<<<<<< Processed data saved to ../output/df_numeric_no_null.csv >>>>>>>>>>

Out-of-domain row removal:
Threshold = 2  standard deviations


INFO:root:Successfully created file df_numeric_reduced.csv in ../output


Number of rows with out-of-domain properties:  0

Sample count before:  10479
Sample count after:  10479
<<<<<<<<<< Processed data saved to ../output/df_numeric_reduced.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/birthdayUnix,user/ageInSeconds
4,4.0,4.0,4.5,1293735206,7.7,4.0,4.5,-2.163082e+09,3.581417e+09
16,4.0,4.0,4.0,1062311123,5.6,4.0,3.0,-3.637188e+08,1.782054e+09
21,4.0,4.0,4.5,1320140421,7.4,4.0,4.5,4.349628e+08,9.833726e+08
25,4.0,4.0,4.0,1318289482,7.4,4.5,5.0,1.545120e+08,1.263823e+09
30,4.0,4.0,4.0,1312403394,7.4,4.5,4.5,5.251644e+08,8.931710e+08
...,...,...,...,...,...,...,...,...,...
49947,4.0,3.5,4.5,1205863037,9.0,4.0,4.0,3.866688e+08,1.031667e+09
49951,3.5,4.5,4.5,1204946359,9.0,4.0,3.5,1.335132e+08,1.284822e+09
49956,3.5,4.0,3.0,1203993243,9.0,3.5,4.0,5.242140e+08,8.941214e+08
49964,4.5,4.0,4.5,1202220776,9.0,4.5,3.5,4.685760e+08,9.497594e+08


Now that the data has been reduced and cleaned of nulls and noise, I move to feature selection.  

First we will split the cleaned dataframe into X and Y to prepare for feature selection using Select K Best with the criteon of f_regression because our attributes and target data values are continuous 

In [12]:
df_numeric_normalized = pd.DataFrame(data_prep.norm_df(df_numeric_reduced))
my_utils.save_to_csv(df_numeric_normalized,'../output', 'df_numeric_normalized.csv' )
df_numeric_normalized

INFO:root:Successfully created file df_numeric_normalized.csv in ../output


<<<<<<<<<< Processed data saved to ../output/df_numeric_normalized.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/birthdayUnix,user/ageInSeconds
4,0.750,0.750,0.875,0.902335,0.184900,0.750,0.875,0.015691,0.984309
16,0.750,0.750,0.750,0.207382,0.130971,0.750,0.500,0.631098,0.368902
21,0.750,0.750,0.875,0.981628,0.177196,0.750,0.875,0.904258,0.095742
25,0.750,0.750,0.750,0.976070,0.177196,0.875,1.000,0.808340,0.191660
30,0.750,0.750,0.750,0.958394,0.177196,0.875,0.875,0.935108,0.064892
...,...,...,...,...,...,...,...,...,...
49947,0.750,0.625,0.875,0.638460,0.218285,0.750,0.750,0.887741,0.112259
49951,0.625,0.875,0.875,0.635707,0.218285,0.750,0.625,0.801158,0.198842
49956,0.625,0.750,0.500,0.632845,0.218285,0.625,0.750,0.934783,0.065217
49964,0.875,0.750,0.875,0.627522,0.218285,0.875,0.625,0.915754,0.084246


After normalization I split my target and non-target attributes into X and Y

In [13]:
# call methods to split target and non-target attributes into X and Y
Y = data_prep.set_y(df_numeric_normalized, target_attribute)
X = data_prep.set_x(df_numeric_normalized, target_attribute)

In [14]:
X.shape

(10479, 8)

In [15]:
Y.shape

(10479,)

In [16]:
# Split data into train and test set with 70% as training data and 30% as test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

# TASK 1: PHASE IV

First model is a multiple linear regression model

In [17]:
# CREATE MODEL 1: MULTI LINEAR REG 

# call regression function without feature selection 
mult_reg_predictions = data_model.mult_regression_model(x_train, y_train, x_test, y_train)

# evaluate model 1
data_model.model_eval(y_test, mult_reg_predictions)

mean_squared_error :  1.11%
mean_absolute_error :  8.02%


Here I see that the mean squared error is 1.11% and the mean absolute error is 8.02%. I think this is a great starting point. 

In [18]:
# CREATE MODEL 2: RIDGE REGRESIION
# Call ridge model creation function passing training data
rdg_predictions = data_model.ridge_reg(x_train, y_train, x_test, y_test)

RGD score: 64.92%
mean_squared_error :  1.11%
mean_absolute_error :  8.01%


The rdg score shows an accuracy measure of 64.92% but the mean square/absolute error calculations are about the same as the linear regression model prior with a 0.01 improvement in the mean_absolute error score. 

In [19]:
# CREATE MODEL 3: Lasso regression model
lasso_pred = data_model.lasso_reg(x_train, y_train, x_test, y_test)

Lasso score: 48.74%
mean_squared_error :  1.62%
mean_absolute_error :  9.75%


So the lasso score was initially -0.07% and this being a negative suggested the model to be performing extremely poorly. However the mean squared error and absolute error were similar, although worse, than the linear and the ridge model. So I realized the problem must be the the alpha value being 0.1 which is too large. After rerunning it with a lower alpha value, of 0.01, the lasso score increased to 48.74% . But that is the most hyper-parameter tuning I'll do for this assignment since we were requested not to fall too deep into that yet.

# TASK 2: ONE HOT ENCODING

Before doing the one-hot encoding. I need to extract the object attributes from the dataframe. I know that even after doing so, I will still be left with some attributes such as birthdates that are saved as objects because their form includes strings. So this can only be corrected by asking for user input to indicate the attribute names that are non-categorical to perform another round of exclusion after this function call strictly by datatype.

In [20]:
# get the original dataframe and extract object variables
df_obj = my_utils.object_df(df)
my_utils.save_to_csv(df_obj, '../output', 'object_data.csv')
df_obj

INFO:root:Successfully created file object_data.csv in ../output


<<<<<<<<<< Processed data saved to ../output/object_data.csv >>>>>>>>>>


,beer/style,beer/name,beer/beerId,beer/brewerId,review/timeStruct,review/text,user/profileName,user/gender,user/birthdayRaw
0,Hefeweizen,Sausa Weizen,47986,10325,"{'isdst': 0, 'mday': 16, 'hour': 20, 'min': 57...",A lot of foam. But a lot.\tIn the smell some b...,stcules,NaN,NaN
1,English Strong Ale,Red Moon,48213,10325,"{'isdst': 0, 'mday': 1, 'hour': 13, 'min': 44,...","Dark red color, light beige foam, average.\tIn...",stcules,NaN,NaN
2,Foreign / Export Stout,Black Horse Black Beer,48215,10325,"{'isdst': 0, 'mday': 1, 'hour': 14, 'min': 10,...","Almost totally black. Beige foam, quite compac...",stcules,NaN,NaN
3,German Pilsener,Sausa Pils,47969,10325,"{'isdst': 0, 'mday': 15, 'hour': 19, 'min': 12...","Golden yellow color. White, compact foam, quit...",stcules,NaN,NaN
4,American Double / Imperial IPA,Cauldron DIPA,64883,1075,"{'isdst': 0, 'mday': 30, 'hour': 18, 'min': 53...","According to the website, the style for the Ca...",johnmichaelsen,Male,"Jun 16, 1901"
...,...,...,...,...,...,...,...,...,...
49995,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,"{'isdst': 0, 'mday': 4, 'hour': 23, 'min': 14,...",Bottled on 5/16/2007 and poured into an Imperi...,Jwale73,NaN,NaN
49996,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,"{'isdst': 0, 'mday': 30, 'hour': 2, 'min': 1, ...",The first beer I put away for cellaring. 12 oz...,bigeyedfish,NaN,NaN
49997,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,"{'isdst': 0, 'mday': 28, 'hour': 3, 'min': 2, ...",12 oz. bottle sampled in a La Fin du Monde tul...,stewdawg,NaN,NaN
49998,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,"{'isdst': 0, 'mday': 27, 'hour': 4, 'min': 34,...","I'm on my second case of Fat Dog, so you kind ...",On2Wheels,NaN,NaN


In [21]:
# User must inpute names of attributes they wish to indicate as non-categorical which have remained in the dataframe
non_cat_col = ['review/timeStruct', 'review/text', 'user/birthdayRaw']
df_categorical = data_prep.remove_non_cat(df_obj, non_cat_col)

In [22]:
df_categorical

,beer/style,beer/name,beer/beerId,beer/brewerId,user/profileName,user/gender
0,Hefeweizen,Sausa Weizen,47986,10325,stcules,NaN
1,English Strong Ale,Red Moon,48213,10325,stcules,NaN
2,Foreign / Export Stout,Black Horse Black Beer,48215,10325,stcules,NaN
3,German Pilsener,Sausa Pils,47969,10325,stcules,NaN
4,American Double / Imperial IPA,Cauldron DIPA,64883,1075,johnmichaelsen,Male
...,...,...,...,...,...,...
49995,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,Jwale73,NaN
49996,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,bigeyedfish,NaN
49997,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,stewdawg,NaN
49998,American Double / Imperial Stout,Stoudt's Fat Dog (Imperial Oatmeal Stout),20539,394,On2Wheels,NaN


NOTE: this is a point where I return to after already doing one-hot-encoding and resulting in a gigantic dataframe that slows down every method of Phase II. So now I'll check on the value counts of each categorical variable and select the top few categorical attribute data points to include in the one-hot-encoding process.

In [23]:
# check value counts for each categorical variable 
def val_count(df):
    top_10_value_counts_names = []

    for col in df.columns:
        print(f"Top 10 value counts for '{col}':")
        value_counts = df[col].value_counts().head(10)
        print(value_counts)
        print("\n")
        top_10_value_counts_names.extend(value_counts.index.tolist())

    return top_10_value_counts_names

In [24]:
top_10_value_counts_names = val_count(df_categorical)

Top 10 value counts for 'beer/style':
American Double / Imperial Stout    5964
American IPA                        4113
American Double / Imperial IPA      3886
Scotch Ale / Wee Heavy              2776
Russian Imperial Stout              2695
American Pale Ale (APA)             2288
American Porter                     2230
Rauchbier                           1938
Rye Beer                            1798
Czech Pilsener                      1501
Name: beer/style, dtype: int64


Top 10 value counts for 'beer/name':
Founders Breakfast Stout                   2502
Founders KBS (Kentucky Breakfast Stout)    1932
Founders Red's Rye PA                      1405
Founders Centennial IPA                    1395
Pilsner Urquell                            1257
Founders Dirty Bastard                     1257
Founders Double Trouble                    1164
Founders Imperial Stout                    1078
Founders Devil Dancer                      1066
Founders Backwoods Bastard                 1020
Na

Seeing this I decide to only use the top 10 value count sample items to perform dummy columns through one-hot encoding for the sake of not making the dataframe too large

In [25]:
df_dummy = my_utils.one_hot_encode_top_values(df_categorical,num_top_values=10 )
df_dummy 

,beer/style_American Double / Imperial Stout,beer/style_American IPA,beer/style_American Double / Imperial IPA,beer/style_Scotch Ale / Wee Heavy,beer/style_Russian Imperial Stout,beer/style_American Pale Ale (APA),beer/style_American Porter,beer/style_Rauchbier,beer/style_Rye Beer,beer/style_Czech Pilsener,...,user/profileName_mikesgroove,user/profileName_MasterSki,user/profileName_oberon,user/profileName_barleywinefiend,user/profileName_akorsak,user/profileName_ChainGangGuy,user/profileName_Thorpe429,user/profileName_BEERchitect,user/gender_Male,user/gender_Female
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


After performing one-hot encoding, I check on the dataframe and I see only zeros in view so just to make sure there does exist values other than zero, I perform a value count check on the first column of the data frame:

Now I can return the target attribute column and save that file as the one-hot encoded dataframe to move on with for Phase II-IV

In [26]:
df_dummy_full = my_utils.return_target_col(df, df_dummy, target_attribute)
df_dummy_full

,beer/style_American Double / Imperial Stout,beer/style_American IPA,beer/style_American Double / Imperial IPA,beer/style_Scotch Ale / Wee Heavy,beer/style_Russian Imperial Stout,beer/style_American Pale Ale (APA),beer/style_American Porter,beer/style_Rauchbier,beer/style_Rye Beer,beer/style_Czech Pilsener,...,user/profileName_MasterSki,user/profileName_oberon,user/profileName_barleywinefiend,user/profileName_akorsak,user/profileName_ChainGangGuy,user/profileName_Thorpe429,user/profileName_BEERchitect,user/gender_Male,user/gender_Female,review/overall
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.0
49996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.0
49997,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
49998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.0


# TASK 2: PHASE II

BEFORE ADJUSTING AMOUNT OF DUMMY COLUMNS I REFLECTED: I note that Phase II for this data takes exceptionally longer than the prior data due to the massive increase of attributes from one-hot encoding. I will still perform a general dataframe description report, quality report, and skew report. 

AFTER LIMITING AMOUNT OF DUMMY COLUMNS: Only report_description even ran succesfully before but now I can actually get a quality and skew report too!

In [27]:
# Print report of dataframe description with format, dimensionality, sample count, attribute count and datatype distribution
# parameters are dataframe, row_ def, and column_def which should recieve associated values of 'samples', and 'attributes' or 'features' to indicate format
data_understanding.report_description(df_dummy_full, 'samples', 'attributes')

GENERATING INITIAL DESCRIPTION OF DATA:

Data dimensions: (50000, 53)
Data has proper formatting with rows=samples and columns=attributes

Sample Count = 
50000
Attribute Count = 
53

Data types that are present:
int64
float64


In [28]:
data_understanding.report_quality(df_dummy_full)

Number of duplicate rows:  48231
Number of rows with missing values: 0

Identifying rows with out-of-domain properties:
Threshold = 2  standard deviations
Number of rows with out-of-domain properties:  741


Now I see a total of 49198 duplicate rows and 1521 rows with out of domain properties that I can remove.

In [29]:
data_understanding.skew_report(df_dummy_full)

Skewness report:
Number of attributes with left skew:  1
Number of attributes with right skew:  52
Number of attributes with no skew/ are symmetric:  0


Ratio of total skewed attributes to symmetric attributes =  53 : 0
Therefor, the data is mostly skewed


Here I recognize that the dummy attributes have caused a great skew to the right wheras the numeric data had a left skew

# TASK 2: PHASE III

In [30]:
# dropping duplicate samples and saving no_duplicate df to output
categorical_no_dup = data_prep.drop_duplicates(df_dummy_full)
my_utils.save_to_csv(categorical_no_dup,'../output', 'categorical_no_dup.csv' )

#dropping null samples and saving numeric data with nulls removed to output

categorical_no_null = data_prep.drop_null(categorical_no_dup)
my_utils.save_to_csv(categorical_no_null,'../output', 'categorical_no_null.csv' )

#dropping rows with out-of-bounds properties and saving reduced numeric data to output

categorical_reduced = pd.DataFrame(data_prep.drop_out_of_domain(categorical_no_null))
my_utils.save_to_csv(categorical_no_null,'../output', 'categorical_reduced.csv' )
categorical_reduced

INFO:root:Successfully created file categorical_no_dup.csv in ../output
INFO:root:Successfully created file categorical_no_null.csv in ../output



Duplicate row removal:
Sample count before:  50000
Sample count after:  1769
<<<<<<<<<< Processed data saved to ../output/categorical_no_dup.csv >>>>>>>>>>

Null row removal:
Sample count before:  1769
Sample count after:  1769
<<<<<<<<<< Processed data saved to ../output/categorical_no_null.csv >>>>>>>>>>

Out-of-domain row removal:
Threshold = 2  standard deviations


INFO:root:Successfully created file categorical_reduced.csv in ../output


Number of rows with out-of-domain properties:  33

Sample count before:  1769
Sample count after:  1736
<<<<<<<<<< Processed data saved to ../output/categorical_reduced.csv >>>>>>>>>>


,beer/style_American Double / Imperial Stout,beer/style_American IPA,beer/style_American Double / Imperial IPA,beer/style_Scotch Ale / Wee Heavy,beer/style_Russian Imperial Stout,beer/style_American Pale Ale (APA),beer/style_American Porter,beer/style_Rauchbier,beer/style_Rye Beer,beer/style_Czech Pilsener,...,user/profileName_MasterSki,user/profileName_oberon,user/profileName_barleywinefiend,user/profileName_akorsak,user/profileName_ChainGangGuy,user/profileName_Thorpe429,user/profileName_BEERchitect,user/gender_Male,user/gender_Female,review/overall
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4.0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,3.0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49932,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.5
49940,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0
49947,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0
49952,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.5


Now that the one-hot-encoded data has been cleaned, I realized all of the samples with nulls were also the duplicate samples.

I then normalize before splitting data to move onto the next phase

In [31]:
categorical_reduced_norm =  data_prep.norm_df(categorical_reduced)
categorical_reduced_norm #visualize values after normalization

,beer/style_American Double / Imperial Stout,beer/style_American IPA,beer/style_American Double / Imperial IPA,beer/style_Scotch Ale / Wee Heavy,beer/style_Russian Imperial Stout,beer/style_American Pale Ale (APA),beer/style_American Porter,beer/style_Rauchbier,beer/style_Rye Beer,beer/style_Czech Pilsener,...,user/profileName_MasterSki,user/profileName_oberon,user/profileName_barleywinefiend,user/profileName_akorsak,user/profileName_ChainGangGuy,user/profileName_Thorpe429,user/profileName_BEERchitect,user/gender_Male,user/gender_Female,review/overall
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.8
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.6
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49932,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9
49940,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.4
49947,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8
49952,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.9


In [32]:
# call methods to split target and non-target attributes into X and Y
Y = data_prep.set_y(categorical_reduced, target_attribute)
X = data_prep.set_x(categorical_reduced, target_attribute)
# Split data into train and test set with 70% as training data and 30% as test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

# TASK 2: PHASE IV

In [33]:
# CREATING MODEL 1: MULTI LINEAR REG WITHOUT FEATURE SELECTION

# passing feature selected X training data, x_test data, y_training and selected features to create model
cat_reg_predictions = data_model.mult_regression_model(x_train, y_train, x_test, y_test)

In [34]:
data_model.model_eval(y_test, cat_reg_predictions)

mean_squared_error :  102.33%
mean_absolute_error :  79.82%


In [35]:
# CREATE MODEL 2: RIDGE REGRESIION

# Call ridge model creation function passing training data
rdg_predictions = data_model.ridge_reg(x_train, y_train, x_test, y_test)

RGD score: 11.72%
mean_squared_error :  102.36%
mean_absolute_error :  79.88%


In [36]:
# CREATE MODEL 3: Lasso regression model
lasso_pred = data_model.lasso_reg(x_train, y_train, x_test, y_test)

Lasso score: 2.78%
mean_squared_error :  112.72%
mean_absolute_error :  84.21%


All of these models performed terribly so I assume the issue lays with my decision to selectively one-hot encode only the top 10 value counts of each attribute whilst excluding the rest of the data values. I believe a better approach would have been to bin the remaining data entries into one column in some joined method. Or perhaps encode the entire set of data but in chunks, and proceed the Phases II-IV in chunks.

In [37]:
# Naive regression

# Calculate the mean of y_train
mean_y_train = np.mean(y_train)

# Create a naive prediction array with the mean value for all samples in the test set
naive_predictions = np.full_like(y_test, mean_y_train)

data_model.model_eval(y_test,naive_predictions)

mean_squared_error :  116.59%
mean_absolute_error :  86.16%


Although the models are pretty bad, theyre atleast better than a naive model!

# TASK 3

Adding the review/text attribute from the original dataframe to the df_numeric dataframe by extracting the string lengths 

In [38]:
df_with_str_len = pd.DataFrame()
df_with_str_len = df_numeric
df_with_str_len['review length'] = my_utils.str_attribute_to_len(df, 'review/text')
my_utils.save_to_csv(df_with_str_len,'../output', 'num_with_str_len.csv' )
df_with_str_len

INFO:root:Successfully created file num_with_str_len.csv in ../output


<<<<<<<<<< Processed data saved to ../output/num_with_str_len.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/birthdayUnix,user/ageInSeconds,review length
0,2.5,1.5,1.5,1234817823,5.0,1.5,2.0,NaN,NaN,262
1,3.0,3.0,3.0,1235915097,6.2,3.0,2.5,NaN,NaN,338
2,3.0,3.0,3.0,1235916604,6.5,3.0,2.5,NaN,NaN,396
3,3.5,2.5,3.0,1234725145,5.0,3.0,3.0,NaN,NaN,401
4,4.0,4.0,4.5,1293735206,7.7,4.0,4.5,-2.163082e+09,3.581417e+09,1145
...,...,...,...,...,...,...,...,...,...,...
49995,4.0,4.0,4.0,1196810078,9.0,4.0,4.0,NaN,NaN,698
49996,3.5,4.0,4.0,1196388069,9.0,4.0,4.0,NaN,NaN,1782
49997,4.0,4.0,4.5,1196218959,9.0,3.5,3.5,NaN,NaN,641
49998,4.0,4.0,4.5,1196138077,9.0,4.0,4.0,NaN,NaN,982


# Task 3: Phase III

In [39]:
# dropping duplicate samples and saving no_duplicate df to output
num_str_no_dup = data_prep.drop_duplicates(df_with_str_len)
my_utils.save_to_csv(num_str_no_dup,'../output', 'num_str_no_dup.csv' )

#dropping null samples and saving numeric data with nulls removed to output

num_str_no_null = data_prep.drop_null(num_str_no_dup)
my_utils.save_to_csv(num_str_no_null,'../output', 'num_str_no_null.csv' )

#dropping rows with out-of-bounds properties and saving reduced numeric data to output

num_str_reduced = pd.DataFrame(data_prep.drop_out_of_domain(num_str_no_null))
my_utils.save_to_csv(num_str_reduced,'../output', 'num_str_reduced.csv' )
num_str_reduced


Duplicate row removal:
Sample count before:  50000
Sample count after:  50000


INFO:root:Successfully created file num_str_no_dup.csv in ../output
INFO:root:Successfully created file num_str_no_null.csv in ../output


<<<<<<<<<< Processed data saved to ../output/num_str_no_dup.csv >>>>>>>>>>

Null row removal:
Sample count before:  50000
Sample count after:  10479
<<<<<<<<<< Processed data saved to ../output/num_str_no_null.csv >>>>>>>>>>

Out-of-domain row removal:
Threshold = 2  standard deviations


INFO:root:Successfully created file num_str_reduced.csv in ../output


Number of rows with out-of-domain properties:  0

Sample count before:  10479
Sample count after:  10479
<<<<<<<<<< Processed data saved to ../output/num_str_reduced.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/birthdayUnix,user/ageInSeconds,review length
4,4.0,4.0,4.5,1293735206,7.7,4.0,4.5,-2.163082e+09,3.581417e+09,1145
16,4.0,4.0,4.0,1062311123,5.6,4.0,3.0,-3.637188e+08,1.782054e+09,394
21,4.0,4.0,4.5,1320140421,7.4,4.0,4.5,4.349628e+08,9.833726e+08,430
25,4.0,4.0,4.0,1318289482,7.4,4.5,5.0,1.545120e+08,1.263823e+09,415
30,4.0,4.0,4.0,1312403394,7.4,4.5,4.5,5.251644e+08,8.931710e+08,904
...,...,...,...,...,...,...,...,...,...,...
49947,4.0,3.5,4.5,1205863037,9.0,4.0,4.0,3.866688e+08,1.031667e+09,503
49951,3.5,4.5,4.5,1204946359,9.0,4.0,3.5,1.335132e+08,1.284822e+09,727
49956,3.5,4.0,3.0,1203993243,9.0,3.5,4.0,5.242140e+08,8.941214e+08,977
49964,4.5,4.0,4.5,1202220776,9.0,4.5,3.5,4.685760e+08,9.497594e+08,444


In [40]:
num_str_reduced_norm = pd.DataFrame(data_prep.norm_df(num_str_reduced))
my_utils.save_to_csv(num_str_reduced_norm,'../output', 'num_str_reduced_norm.csv' )
num_str_reduced_norm

INFO:root:Successfully created file num_str_reduced_norm.csv in ../output


<<<<<<<<<< Processed data saved to ../output/num_str_reduced_norm.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/birthdayUnix,user/ageInSeconds,review length
4,0.750,0.750,0.875,0.902335,0.184900,0.750,0.875,0.015691,0.984309,0.263764
16,0.750,0.750,0.750,0.207382,0.130971,0.750,0.500,0.631098,0.368902,0.090762
21,0.750,0.750,0.875,0.981628,0.177196,0.750,0.875,0.904258,0.095742,0.099056
25,0.750,0.750,0.750,0.976070,0.177196,0.875,1.000,0.808340,0.191660,0.095600
30,0.750,0.750,0.750,0.958394,0.177196,0.875,0.875,0.935108,0.064892,0.208247
...,...,...,...,...,...,...,...,...,...,...
49947,0.750,0.625,0.875,0.638460,0.218285,0.750,0.750,0.887741,0.112259,0.115872
49951,0.625,0.875,0.875,0.635707,0.218285,0.750,0.625,0.801158,0.198842,0.167473
49956,0.625,0.750,0.500,0.632845,0.218285,0.625,0.750,0.934783,0.065217,0.225063
49964,0.875,0.750,0.875,0.627522,0.218285,0.875,0.625,0.915754,0.084246,0.102281


In [41]:
# call methods to split target and non-target attributes into X and Y
Y = data_prep.set_y(num_str_reduced_norm, target_attribute)
X = data_prep.set_x(num_str_reduced_norm, target_attribute)
# Split data into train and test set with 70% as training data and 30% as test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

# TASK 3: PHASE IV

In [42]:
# CREATING MODEL 1: MULTI LINEAR REG WITHOUT FEATURE SELECTION

# passing feature selected X training data, x_test data, y_training and selected features to create model
num_str_reg_pred = data_model.mult_regression_model(x_train, y_train, x_test, y_test)
data_model.model_eval(y_test, num_str_reg_pred)

mean_squared_error :  1.11%
mean_absolute_error :  8.02%


In [43]:
# CREATE MODEL 2: RIDGE REGRESIION

# Call ridge model creation function passing training data
num_str_reg_pred = data_model.ridge_reg(x_train, y_train, x_test, y_test)

RGD score: 64.94%
mean_squared_error :  1.11%
mean_absolute_error :  8.01%


In [44]:
# CREATE MODEL 3: Lasso regression model
lasso_pred = data_model.lasso_reg(x_train, y_train, x_test, y_test)

Lasso score: 48.74%
mean_squared_error :  1.62%
mean_absolute_error :  9.75%


Quantitative assessment: All the regression models built from the numeric data that included the length of the reviews performed exactly the same as the models built from only the numeric data. This exposed the irrelevance of review/text string length on predicting the target attribute. I hope to find another irrelevant attribute in the next task. 

# TASK 4: ABLATION STUDY

I will try to test the numeric data excluding the user/birthday Unix attribute because if age is a factor in beer reviews at all, it is probably more represented in the age by seconds rather than this feature.

In [45]:
df_num_no_birthday = pd.DataFrame(num_str_reduced_norm.drop('user/birthdayUnix', axis=1))
df_num_no_birthday = df_num_no_birthday.drop('review length', axis = 1)
my_utils.save_to_csv(df_num_no_birthday,'../output', 'df_num_no_birthday.csv' )
df_num_no_birthday

INFO:root:Successfully created file df_num_no_birthday.csv in ../output


<<<<<<<<<< Processed data saved to ../output/df_num_no_birthday.csv >>>>>>>>>>


,review/appearance,review/palate,review/taste,review/timeUnix,beer/ABV,review/overall,review/aroma,user/ageInSeconds
4,0.750,0.750,0.875,0.902335,0.184900,0.750,0.875,0.984309
16,0.750,0.750,0.750,0.207382,0.130971,0.750,0.500,0.368902
21,0.750,0.750,0.875,0.981628,0.177196,0.750,0.875,0.095742
25,0.750,0.750,0.750,0.976070,0.177196,0.875,1.000,0.191660
30,0.750,0.750,0.750,0.958394,0.177196,0.875,0.875,0.064892
...,...,...,...,...,...,...,...,...
49947,0.750,0.625,0.875,0.638460,0.218285,0.750,0.750,0.112259
49951,0.625,0.875,0.875,0.635707,0.218285,0.750,0.625,0.198842
49956,0.625,0.750,0.500,0.632845,0.218285,0.625,0.750,0.065217
49964,0.875,0.750,0.875,0.627522,0.218285,0.875,0.625,0.084246


In [46]:
# call methods to split target and non-target attributes into X and Y
Y = data_prep.set_y(num_str_reduced_norm, target_attribute)
X = data_prep.set_x(num_str_reduced_norm, target_attribute)
# Split data into train and test set with 70% as training data and 30% as test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

In [47]:
# CREATING MODEL 1: MULTI LINEAR REG WITHOUT FEATURE SELECTION

# passing feature selected X training data, x_test data, y_training and selected features to create model
num_str_reg_pred = data_model.mult_regression_model(x_train, y_train, x_test, y_test)
data_model.model_eval(y_test, num_str_reg_pred)

mean_squared_error :  1.11%
mean_absolute_error :  8.02%


In [48]:
# CREATE MODEL 2: RIDGE REGRESIION

# Call ridge model creation function passing training data
num_str_reg_pred = data_model.ridge_reg(x_train, y_train, x_test, y_test)

RGD score: 64.94%
mean_squared_error :  1.11%
mean_absolute_error :  8.01%


In [49]:
# CREATE MODEL 3: Lasso regression model
lasso_pred = data_model.lasso_reg(x_train, y_train, x_test, y_test)

Lasso score: 48.74%
mean_squared_error :  1.62%
mean_absolute_error :  9.75%


Assessment: The feature user/birthday Unix also had no impact on the models and thus was proven to be uninformative